In [2]:
import pandas as pd
import numpy as np
import pickle
import json

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from scipy.stats import shapiro, kstest, normaltest,skew

import matplotlib.pyplot as plt
import seaborn as sns


### problem statement

In [ ]:
to predict mobile prices based on the features
Features:
    

### Data Gathering

In [3]:
df = pd.read_csv("mobile_price.csv")
df

,Name,Link,Make,System,Memory,RAM,Display,Display_Quality,Camera,Battery,Sim,Screen Refresh Rate,Price
0,realme Realme C21 64GB Mavi Cep Telefonu (Real...,https://www.trendyol.com/realme/realme-c21-64g...,realme,Android,64 GB,3 GB,LCD,HD+,10 - 15 MP,4500-5000 mAh,4G,60 Hz,2388.0
1,Apple iPhone 11 64GB Beyaz Cep Telefonu (Apple...,https://www.trendyol.com/apple/iphone-11-64gb-...,Apple,iOS,64 GB,4 GB,LCD,HD+,10 - 15 MP,3000-3500 mAh,4.5G,60 Hz,9799.0
2,POCO X3 Pro 8GB + 256GB Siyah Cep Telefonu Xia...,https://www.trendyol.com/poco/x3-pro-8gb-256gb...,POCO,Android,256 GB,8 GB,LCD,FHD+,20 MP ve üstü,5000-6000 mAh,4.5G,120 Hz,5619.0
3,Apple iPhone 11 64GB Siyah Cep Telefonu (Apple...,https://www.trendyol.com/apple/iphone-11-64gb-...,Apple,iOS,64 GB,4 GB,LCD,HD+,10 - 15 MP,3000-3500 mAh,4.5G,60 Hz,9789.0
4,Samsung Galaxy Note20 Ultra 256GB Mystic Bronz...,https://www.trendyol.com/samsung/galaxy-note20...,Samsung,Android,256 GB,8 GB,AMOLED,FHD+,20 MP ve üstü,4500-5000 mAh,4.5G,120 Hz,12199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,Tecno Pova Tecno Spark6go-64-mblue 13mp Spark6...,https://www.trendyol.com/tecno-pova/tecno-spar...,Tecno,Android,64 GB,4 GB,LCD,NaN,NaN,2000-3000 mAh,4.5G,60 Hz,3299.0
935,Xiaomi Redmi Note 9 128 Gb Cep Telefonu 123456...,https://www.trendyol.com/xiaomi/redmi-note-9-1...,Xiaomi,Android,128 GB,4 GB,NaN,FHD+,20 MP ve üstü,3500-4000 mAh,3G,NaN,2299.0
936,Reeder P13 Blue Max 2021 128 Gb 4 Gb Ram Türki...,https://www.trendyol.com/reeder/p13-blue-max-2...,Reeder,Android,128 GB,4 GB,LCD,NaN,10 - 15 MP,4500-5000 mAh,4.5G,60 Hz,3570.0
937,Apple iPhone 13 Mini 512GB Mavi Cep Telefonu (...,https://www.trendyol.com/apple/iphone-13-mini-...,Apple,iOS,512 GB,4 GB,NaN,NaN,NaN,2000-3000 mAh,4.5G,NaN,2949.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939 entries, 0 to 938
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 939 non-null    object 
 1   Link                 938 non-null    object 
 2   Make                 938 non-null    object 
 3   System               938 non-null    object 
 4   Memory               938 non-null    object 
 5   RAM                  938 non-null    object 
 6   Display              568 non-null    object 
 7   Display_Quality      603 non-null    object 
 8   Camera               617 non-null    object 
 9   Battery              938 non-null    object 
 10  Sim                  934 non-null    object 
 11  Screen Refresh Rate  610 non-null    object 
 12  Price                937 non-null    float64
dtypes: float64(1), object(12)
memory usage: 95.5+ KB


In [4]:
df = df.drop(['Name', 'Link'], axis = 1)
df

,Make,System,Memory,RAM,Display,Display_Quality,Camera,Battery,Sim,Screen Refresh Rate,Price
0,realme,Android,64 GB,3 GB,LCD,HD+,10 - 15 MP,4500-5000 mAh,4G,60 Hz,2388.0
1,Apple,iOS,64 GB,4 GB,LCD,HD+,10 - 15 MP,3000-3500 mAh,4.5G,60 Hz,9799.0
2,POCO,Android,256 GB,8 GB,LCD,FHD+,20 MP ve üstü,5000-6000 mAh,4.5G,120 Hz,5619.0
3,Apple,iOS,64 GB,4 GB,LCD,HD+,10 - 15 MP,3000-3500 mAh,4.5G,60 Hz,9789.0
4,Samsung,Android,256 GB,8 GB,AMOLED,FHD+,20 MP ve üstü,4500-5000 mAh,4.5G,120 Hz,12199.0
...,...,...,...,...,...,...,...,...,...,...,...
934,Tecno,Android,64 GB,4 GB,LCD,NaN,NaN,2000-3000 mAh,4.5G,60 Hz,3299.0
935,Xiaomi,Android,128 GB,4 GB,NaN,FHD+,20 MP ve üstü,3500-4000 mAh,3G,NaN,2299.0
936,Reeder,Android,128 GB,4 GB,LCD,NaN,10 - 15 MP,4500-5000 mAh,4.5G,60 Hz,3570.0
937,Apple,iOS,512 GB,4 GB,NaN,NaN,NaN,2000-3000 mAh,4.5G,NaN,2949.0


### EDA

#### 1.Make

In [7]:
df['Make'].value_counts()

Make
Samsung           213
Apple             160
Xiaomi            146
General Mobile     55
Oppo               45
Reeder             44
Huawei             36
realme             35
Tecno              30
Infinix            21
POCO               19
Hiking             17
Casper             16
TCL                14
LG                 13
Vivo               11
VESTEL              8
Elephone            8
Alcatel             8
Omix                7
Ulefone             6
Honor               5
Oneplus             4
Trident             4
Meizu               3
Doogee              2
Htc                 1
Motorola            1
Smart               1
Nokia               1
Oukitel             1
LENOVO              1
Cat                 1
Sunny               1
Name: count, dtype: int64

In [8]:
df['Make'].isna().sum()

1

In [9]:
df['Make'].mode()[0]

'Samsung'

In [5]:
df['Make'].fillna(df['Make'].mode()[0], inplace=True)

In [6]:
df = pd.get_dummies(df,columns= ['Make'], dtype=int)

In [7]:
df

,System,Memory,RAM,Display,Display_Quality,Camera,Battery,Sim,Screen Refresh Rate,Price,...,Make_Smart,Make_Sunny,Make_TCL,Make_Tecno,Make_Trident,Make_Ulefone,Make_VESTEL,Make_Vivo,Make_Xiaomi,Make_realme
0,Android,64 GB,3 GB,LCD,HD+,10 - 15 MP,4500-5000 mAh,4G,60 Hz,2388.0,...,0,0,0,0,0,0,0,0,0,1
1,iOS,64 GB,4 GB,LCD,HD+,10 - 15 MP,3000-3500 mAh,4.5G,60 Hz,9799.0,...,0,0,0,0,0,0,0,0,0,0
2,Android,256 GB,8 GB,LCD,FHD+,20 MP ve üstü,5000-6000 mAh,4.5G,120 Hz,5619.0,...,0,0,0,0,0,0,0,0,0,0
3,iOS,64 GB,4 GB,LCD,HD+,10 - 15 MP,3000-3500 mAh,4.5G,60 Hz,9789.0,...,0,0,0,0,0,0,0,0,0,0
4,Android,256 GB,8 GB,AMOLED,FHD+,20 MP ve üstü,4500-5000 mAh,4.5G,120 Hz,12199.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,Android,64 GB,4 GB,LCD,NaN,NaN,2000-3000 mAh,4.5G,60 Hz,3299.0,...,0,0,0,1,0,0,0,0,0,0
935,Android,128 GB,4 GB,NaN,FHD+,20 MP ve üstü,3500-4000 mAh,3G,NaN,2299.0,...,0,0,0,0,0,0,0,0,1,0
936,Android,128 GB,4 GB,LCD,NaN,10 - 15 MP,4500-5000 mAh,4.5G,60 Hz,3570.0,...,0,0,0,0,0,0,0,0,0,0
937,iOS,512 GB,4 GB,NaN,NaN,NaN,2000-3000 mAh,4.5G,NaN,2949.0,...,0,0,0,0,0,0,0,0,0,0


#### 2.System

In [17]:
df['System'].value_counts()

System
Android    777
iOS        161
Name: count, dtype: int64

In [23]:
df['System'].isna().sum()

1

In [88]:
df['System'].mode()[0]

'Android'

In [8]:
df['System'].fillna(df['System'].mode()[0], inplace=True)

In [9]:
df['System'].replace({'Android' : 1, 'iOS':2}, inplace=True)

In [10]:
System_data = {'Android' : 1, 'iOS':2}

#### 3.Memory

In [91]:
df['Memory'].value_counts().to_dict()

{'128 GB': 393,
 '64 GB': 245,
 '32 GB': 120,
 '256 GB': 104,
 '16 GB': 32,
 '512 GB': 25,
 '1 TB': 8,
 '8 GB': 6,
 '4 GB': 2,
 '512 MB': 2,
 'Yok': 1}

In [22]:
df['Memory'].isna().sum()

1

In [92]:
df['Memory'].mode()[0]

'128 GB'

In [11]:
df['Memory'].fillna(df['Memory'].mode()[0], inplace=True)

In [12]:
df['Memory'].replace({'128 GB':7 ,
                        '64 GB':6 ,
                        '32 GB': 5,
                         '256 GB': 8,
                         '16 GB': 4,
                         '512 GB': 9,
                         '1 TB': 10,
                         '8 GB': 3,
                         '4 GB': 2,
                         '512 MB':1,
                         'Yok': 0}, inplace=True)

In [13]:
Memory_label = {'128 GB':7 ,
                        '64 GB':6 ,
                        '32 GB': 5,
                         '256 GB': 8,
                         '16 GB': 4,
                         '512 GB': 9,
                         '1 TB': 10,
                         '8 GB': 3,
                         '4 GB': 2,
                         '512 MB':1,
                         'Yok': 0}

#### 3.RAM

In [96]:
df['RAM'].value_counts().to_dict()

{'4 GB': 373,
 '6 GB': 190,
 '8 GB': 153,
 '3 GB': 109,
 '2 GB': 84,
 '1 GB': 16,
 '12 GB': 12,
 '32 GB': 1}

In [97]:
df['RAM'].isna().sum()

1

In [98]:
df['RAM'].mode()[0]

'4 GB'

In [14]:
df['RAM'].fillna(df['RAM'].mode()[0], inplace=True)

In [15]:
df['RAM'].replace({'4 GB': 4,
                     '6 GB': 6,
                     '8 GB': 7,
                     '3 GB': 3,
                     '2 GB': 2,
                     '1 GB': 1,
                     '12 GB': 8,
                     '32 GB': 9}, inplace=True)

In [16]:
RAM_label = {'4 GB': 4,
                     '6 GB': 6,
                     '8 GB': 7,
                     '3 GB': 3,
                     '2 GB': 2,
                     '1 GB': 1,
                     '12 GB': 8,
                     '32 GB': 9}

In [102]:
df['Display'].value_counts().to_dict()

{'LCD': 367, 'AMOLED': 140, 'OLED': 59, 'LED': 2}

In [103]:
df['Display'].isna().sum()

371

In [104]:
df['Display'].mode()[0]

'LCD'

In [17]:
df['Display'].fillna(df['Display'].mode()[0], inplace=True)

In [18]:
df['Display'].replace({'LCD': 1, 'AMOLED': 4, 'OLED': 3, 'LED': 2}, inplace=True)

In [19]:
Display_label = {'LCD': 1, 'AMOLED': 4, 'OLED': 3, 'LED': 2}

In [108]:
df['Display_Quality'].value_counts().to_dict()

{'FHD+': 294, 'HD+': 258, 'HD': 24, 'QHD+': 23, 'QHD': 3, 'UHD+': 1}

In [109]:
df['Display_Quality'].isna().sum()

336

In [110]:
df['Display_Quality'].mode()[0]

'FHD+'

In [21]:
df['Display_Quality'].fillna(df['Display_Quality'].mode()[0], inplace=True)

In [22]:
df['Display_Quality'].replace({'FHD+': 3, 'HD+': 2, 'HD': 1, 'QHD+': 5, 'QHD': 4, 'UHD+': 6}, inplace=True)

In [23]:
Display_Quality_label = {'FHD+': 3, 'HD+': 2, 'HD': 1, 'QHD+': 5, 'QHD': 4, 'UHD+': 6}

In [114]:
df['Camera'].value_counts().to_dict()

{'20 MP ve üstü': 277,
 '10 - 15 MP': 264,
 '15 - 20 MP': 45,
 '5 - 10 MP': 28,
 '5 MP ve altı': 3}

In [115]:
df['Camera'].isna().sum()

322

In [116]:
df['Camera'].isna().mean()

0.34291799787007454

In [117]:
df['Camera'].mode()[0]

'20 MP ve üstü'

In [24]:
df['Camera'].fillna(df['Camera'].mode()[0], inplace=True)

In [25]:
df['Camera'].replace({'20 MP ve üstü': 5,
                         '10 - 15 MP':4,
                         '15 - 20 MP': 3,
                         '5 - 10 MP': 2,
                         '5 MP ve altı': 1}, inplace = True)

In [26]:
Camera_label = {'20 MP ve üstü': 5,
                         '10 - 15 MP':4,
                         '15 - 20 MP': 3,
                         '5 - 10 MP': 2,
                         '5 MP ve altı': 1}


In [28]:
df['Battery'].value_counts().to_dict()

{'4500-5000 mAh': 247,
 '5000-6000 mAh': 185,
 '4000-4500 mAh': 149,
 '3500-4000 mAh': 130,
 '3000-3500 mAh': 107,
 '2000-3000 mAh': 86,
 '1000-2000 mAh': 18,
 '800-1000 mAh': 11,
 '6000-7000 mAh': 6}

In [121]:
df['Battery'].isna().sum()

1

In [72]:
df['Battery'].mode()[0]

'4500-5000 mAh'

In [27]:
df['Battery'].fillna(df['Battery'].mode()[0], inplace=True)

In [29]:
df['Battery'].replace({'4500-5000 mAh': 8,
                         '5000-6000 mAh': 7,
                         '4000-4500 mAh': 6,
                         '3500-4000 mAh': 5,
                         '3000-3500 mAh': 4,
                         '2000-3000 mAh': 3,
                         '1000-2000 mAh': 2,
                         '800-1000 mAh': 1,
                         '6000-7000 mAh': 9}, inplace =True)

In [30]:
Battery_label = {'4500-5000 mAh': 8,
                        '5000-6000 mAh': 7,
                         '4000-4500 mAh': 6,
                         '3500-4000 mAh': 5,
                         '3000-3500 mAh': 4,
                         '2000-3000 mAh': 3,
                         '1000-2000 mAh': 2,
                         '800-1000 mAh': 1,
                         '6000-7000 mAh': 9}

In [125]:
df['Sim'].value_counts().to_dict()

{'4.5G': 682, '5G': 169, '4G': 58, '3G': 18, '2G': 7}

In [78]:
df['Sim'].isna().sum()

5

In [79]:
df['Sim'].mode()[0]

'4.5G'

In [31]:
df['Sim'].fillna(df['Sim'].mode()[0], inplace=True)

In [32]:
df['Sim'].replace({'4.5G': 5, '5G': 4, '4G': 3, '3G': 2, '2G': 1}, inplace=True)

In [33]:
Sim_label= {'4.5G': 5, '5G': 4, '4G': 3, '3G': 2, '2G': 1}

#### 10.Screen Refresh Rate

In [130]:
df['Screen Refresh Rate'].value_counts().to_dict()

{'60 Hz': 501,
 '120 Hz': 56,
 '90 Hz': 44,
 '60 Hz altı': 6,
 '144 Hz': 2,
 '144 Hz üstü': 1}

In [131]:
df['Screen Refresh Rate'].isna().sum()

329

In [133]:
df['Screen Refresh Rate'].mode()[0]

'60 Hz'

In [34]:
df['Screen Refresh Rate'].fillna(df['Screen Refresh Rate'].mode()[0], inplace=True)

In [35]:
df['Screen Refresh Rate'].replace({'60 Hz': 1,
                                     '120 Hz': 3,
                                     '90 Hz': 2,
                                     '60 Hz altı':5,
                                     '144 Hz': 4,
                                     '144 Hz üstü': 6}, inplace=True)

In [36]:
Screen_Refresh_Rate_label= {'60 Hz': 1,
                    '120 Hz': 3,
                    '90 Hz': 2,
                     '60 Hz altı':5,
                     '144 Hz': 4,
                    '144 Hz üstü': 6}

In [141]:
df['Price'].isna().sum()

2

In [145]:
df['Price'].mean()

6692.733191035219

In [37]:
df['Price'].fillna(df['Price'].mean(), inplace=True)

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939 entries, 0 to 938
Data columns (total 44 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   System               939 non-null    int64  
 1   Memory               939 non-null    int64  
 2   RAM                  939 non-null    int64  
 3   Display              939 non-null    int64  
 4   Display_Quality      939 non-null    int64  
 5   Camera               939 non-null    int64  
 6   Battery              939 non-null    int64  
 7   Sim                  939 non-null    int64  
 8   Screen Refresh Rate  939 non-null    int64  
 9   Price                939 non-null    float64
 10  Make_Alcatel         939 non-null    int32  
 11  Make_Apple           939 non-null    int32  
 12  Make_Casper          939 non-null    int32  
 13  Make_Cat             939 non-null    int32  
 14  Make_Doogee          939 non-null    int32  
 15  Make_Elephone        939 non-null    int

### Model Training


In [38]:
x = df.drop('Price', axis = 1)
y= df['Price']

In [39]:
x_train, x_test,y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=1)

In [40]:
lin_reg = LinearRegression()
lin_reg.fit(x_train,y_train)

LinearRegression()

### Evaluation

#### Testing Model

In [41]:
y_test_pred = lin_reg.predict(x_test)

mse = mean_squared_error(y_test,y_test_pred)
print("MSE : ", mse)

rmse = np.sqrt(mse)
print("RMSE : ", rmse)

mae = mean_absolute_error(y_test, y_test_pred)
print("MAE : ", mae)

r2value = r2_score(y_test,y_test_pred)
print("R2 value is : ", r2value)

MSE :  32447880.92171937
RMSE :  5696.304145822918
MAE :  3764.480326710771
R2 value is :  0.19737777727116101


#### Training Model

In [42]:
y_train_pred = lin_reg.predict(x_train)

mse = mean_squared_error(y_train,y_train_pred)
print("MSE : ", mse)

rmse = np.sqrt(mse)
print("RMSE : ", rmse)

mae = mean_absolute_error(y_train,y_train_pred)
print("MAE : ", mae)

r2value = r2_score(y_train,y_train_pred)
print("R2 value is : ", r2value)

MSE :  24311782.369934812
RMSE :  4930.697959714711
MAE :  3277.2207414450245
R2 value is :  0.3488488977134452


### Residuals

In [43]:
residuals = y_test-y_test_pred
residuals

604     9709.166110
527    -1493.091532
893    -6317.704598
195    -1687.350321
422    -1428.165831
           ...     
528      200.659575
311    12802.854342
57     -4123.224740
495     2139.624231
837     8369.447729
Name: Price, Length: 282, dtype: float64

#### single row testing

In [44]:
x_test[151:152]

,System,Memory,RAM,Display,Display_Quality,Camera,Battery,Sim,Screen Refresh Rate,Make_Alcatel,...,Make_Smart,Make_Sunny,Make_TCL,Make_Tecno,Make_Trident,Make_Ulefone,Make_VESTEL,Make_Vivo,Make_Xiaomi,Make_realme
457,1,6,4,1,3,5,8,5,1,0,...,0,0,0,0,0,0,0,0,1,0


In [46]:
lin_reg.predict(x_test[151:152])[0]

4225.889858748076

In [47]:
column_names= x.columns.to_list()
column_names

['System',
 'Memory',
 'RAM',
 'Display',
 'Display_Quality',
 'Camera',
 'Battery',
 'Sim',
 'Screen Refresh Rate',
 'Make_Alcatel',
 'Make_Apple',
 'Make_Casper',
 'Make_Cat',
 'Make_Doogee',
 'Make_Elephone',
 'Make_General Mobile',
 'Make_Hiking',
 'Make_Honor',
 'Make_Htc',
 'Make_Huawei',
 'Make_Infinix',
 'Make_LENOVO',
 'Make_LG',
 'Make_Meizu',
 'Make_Motorola',
 'Make_Nokia',
 'Make_Omix',
 'Make_Oneplus',
 'Make_Oppo',
 'Make_Oukitel',
 'Make_POCO',
 'Make_Reeder',
 'Make_Samsung',
 'Make_Smart',
 'Make_Sunny',
 'Make_TCL',
 'Make_Tecno',
 'Make_Trident',
 'Make_Ulefone',
 'Make_VESTEL',
 'Make_Vivo',
 'Make_Xiaomi',
 'Make_realme']

In [48]:
lin_reg.n_features_in_

43

In [49]:
System='Android'
Memory='128 GB'
RAM='4 GB'
Display='LCD'
Display_Quality='HD'
Camera='20 MP ve üstü'
Battery='4500-5000 mAh'
Sim='4.5G'
Screen_Refresh_Rate='60 Hz'

In [50]:
Mobileprice = {'System' : System_data ,'Memory': Memory_label, 'RAM' :RAM_label, 'Display' :Display_label,
               'Display_Quality' : Display_Quality_label, 'Camera' :Camera_label, 'Battery' :Battery_label,
                    'Sim' : Sim_label, 'Screen_Refresh_Rate' : Screen_Refresh_Rate_label, 'Column_names' : column_names}

In [51]:
with open("Mobile_Price.pkl", "wb") as f:
    pickle.dump(lin_reg,f)

In [52]:
with open("Mobile_price.json", "w") as f:
    json.dump(Mobileprice, f)